In [1]:
!wget https://datahack-prod.s3.amazonaws.com/test_file/test_Yix80N0.csv

--2020-05-01 18:08:51--  https://datahack-prod.s3.amazonaws.com/test_file/test_Yix80N0.csv
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.64.48
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.64.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 465312 (454K) [text/csv]
Saving to: ‘test_Yix80N0.csv’

test_Yix80N0.csv    100%[===================>] 454.41K   487KB/s    in 0.9s    

2020-05-01 18:08:53 (487 KB/s) - ‘test_Yix80N0.csv’ saved [465312/465312]



In [2]:
!wget https://datahack-prod.s3.amazonaws.com/train_file/train_8wry4cB.csv

--2020-05-01 18:08:55--  https://datahack-prod.s3.amazonaws.com/train_file/train_8wry4cB.csv
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.64.24
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.64.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1138290 (1.1M) [text/csv]
Saving to: ‘train_8wry4cB.csv’

train_8wry4cB.csv   100%[===================>]   1.08M   794KB/s    in 1.4s    

2020-05-01 18:08:58 (794 KB/s) - ‘train_8wry4cB.csv’ saved [1138290/1138290]



In [0]:
# !pip install wget
import wget

In [0]:
wget.download('https://datahack-prod.s3.amazonaws.com/test_file/test_Yix80N0.csv')
wget.download('https://datahack-prod.s3.amazonaws.com/train_file/train_8wry4cB.csv')

In [0]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import seaborn as sns

In [0]:
data_train = pd.read_csv('train_8wry4cB.csv')
data_test = pd.read_csv('test_Yix80N0.csv')

In [0]:
train_bk = data_train.copy(deep = True)
test_bk = data_test.copy(deep = True)

In [0]:
def clean(data):
    
    data.startTime = pd.to_datetime(data.startTime)
    data['week'] = data.startTime.dt.weekofyear
    data.drop(columns=['endTime','startTime','ProductList','session_id'],inplace = True)
    
    if 'gender' in data.columns:
        data.gender.replace({'female':1, 'male':0},inplace= True)
    
    return data

In [0]:
def create_item_list(dataset, train = True, col = None):
    items_array = pd.Series()
    import re
    
    if train:
        for list_ in dataset['ProductList']:
            s = re.split(r'[^\w]',list_)
            items_ = np.array(s)
            items_ = items_.ravel()
            items_array = np.append(items_array,items_)

        items_list = np.unique(items_array)

        df = pd.DataFrame(columns=items_list[1:], index = dataset.index, data = 0)
    else:
        items_list = col
        df = pd.DataFrame(columns=items_list[1:], index = dataset.index, data = 0)
        
        
        
    for index, list_ in enumerate(dataset['ProductList']):
        s = re.split(r'[^\w]',list_)
        items_ = np.array(s)
        items_ = items_.ravel()
        for item in items_:
            if item in df.columns:
                if item != '':
                    df[item][index] += 1
    
    dataset_ = clean(dataset)
    data = pd.concat([df,dataset_] ,axis =1)
    
    return data,items_list

In [7]:
df_train,items = create_item_list(train_bk)
df_test,_ = create_item_list(test_bk , train = False ,col=items)

CPU times: user 45.1 s, sys: 2.22 s, total: 47.3 s
Wall time: 47.3 s


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
!pip install catboost
from catboost import CatBoostClassifier


In [0]:
X = df_train.drop(columns='gender').values
y = df_train.values

##Base models

In [0]:
X_train,X_val,y_train,y_val = train_test_split(X, y, random_state = 6)

In [0]:
%%time
clf = LogisticRegression()

clf.fit(X_train,y_train)

y_predict = clf.predict(X_val)
print(clf.score(X_train,y_train))
print(confusion_matrix(y_val,y_predict))
print('accuracy_score :' ,accuracy_score(y_val,y_predict))

In [0]:
%%time
clf = ExtraTreesClassifier()

clf.fit(X_train,y_train)

y_predict = clf.predict(X_val)
print(clf.score(X_train,y_train))
print(confusion_matrix(y_val,y_predict))
print('accuracy_score :' ,accuracy_score(y_val,y_predict))

In [0]:
%%time
clf = RandomForestClassifier()

clf.fit(X_train,y_train)

y_predict = clf.predict(X_val)
print(clf.score(X_train,y_train))
print(confusion_matrix(y_val,y_predict))
print('accuracy_score :' ,accuracy_score(y_val,y_predict))

In [0]:
%%time
clf = DecisionTreeClassifier()

clf.fit(X_train,y_train)

y_predict = clf.predict(X_val)
print(clf.score(X_train,y_train))
print(confusion_matrix(y_val,y_predict))
print('accuracy_score :' ,accuracy_score(y_val,y_predict))

In [0]:
# from catboost import CatBoostClassifier
cat = CatBoostClassifier(loss_function='Logloss', 
                         eval_metric='Accuracy', 
                         depth=10,
                         random_seed=42, 
                         iterations=100, 
                         learning_rate=0.01,
                         leaf_estimation_iterations=1,
                         l2_leaf_reg=1,
                         bootstrap_type='Bayesian', 
                         bagging_temperature=1, 
                         random_strength=1,
                         od_type='Iter', 
                         od_wait=None)
cat.fit(X_train, y_train, verbose=10,
        use_best_model=True,
        eval_set=[(X_train, y_train),(X_val, y_val)],
        plot=False)

y_predict = cat.predict(X_val)
print(confusion_matrix(y_val,y_predict))
print('accuracy_score :' ,accuracy_score(y_val,y_predict))

In [0]:
def build_model():
  model = keras.Sequential([
    layers.Dense(16960, activation='tanh', input_shape=(16960,)),
    # layers.Dense(1, activation='relu'),
    # layers.Dense(20, activation= 'relu'),
    # layers.Dense(20, activation = 'relu'),
    # layers.Dense(10, activation = 'relu'),
    # layers.Dense(10, activation = 'tanh'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=True)
  optimizer_adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=True)
  optimizer_adamax = tf.keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)



#   model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['acc'])
  model.compile(loss='binary_crossentropy', optimizer = optimizer_adamax , metrics=['accuracy'])

  return model

In [0]:
model_os = build_model()

In [0]:
EPOCHS = 2000

history = model_os.fit(data_x.values, data_y,
  epochs=EPOCHS, validation_split = 0.2, verbose=1,
  use_multiprocessing = True,workers = -1,
  batch_size = 1000)

In [0]:
%%time
clf = RandomForestClassifier()

clf.fit(X,y)

y_predict = clf.predict(data_test)

In [0]:
out = pd.DataFrame(y_predict,columns = ['gender'])
out['session_id'] = data_test.session_id
out.gender.replace({1:'female',0:'male'})

In [0]:
out.to_csv('rgd_rf.csv')